### 実行環境
注1) tensorflowに内包されたkerasを使用することで、model.fitの入力にdatasetのiteratorを使用可能である。
https://github.com/tensorflow/tensorflow/issues/20698

注2) tensorflowのDataset APIを活用する場合、tensorflow内のkerasを利用するとエラーが出る不具合がある。
```
ValueError: Cannot take the length of shape with unknown rank.
```
https://stackoverflow.com/questions/53851793/valueerror-cannot-take-the-length-of-shape-with-unknown-rank

In [13]:
import tensorflow as tf
print(tf.__version__)

import keras
print(keras.__version__)

1.14.0
2.2.4


In [0]:
from keras import backend as K
sess = tf.Session()
K.set_session( sess )

In [0]:
import os
os.environ[ 'TF_CPP_MIN_LOG_LEVEL'] = '2'

### GoogleDriveをマウントする。

In [16]:
import os
if os.name == 'nt':
    print('OS is Windows: PASS mount google drive')
    g_dir_work = '../colab/'
else:
    from google.colab import drive
    drive.mount('/content/drive')
    g_dir_work = '/content/drive/My Drive/colab/'

# check mount point
print('\n<< Display work dir >>')
for file in os.listdir(g_dir_work):
    print( 'file/dir : ', file)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

<< Display work dir >>
file/dir :  mylib
file/dir :  test.txt
file/dir :  data
file/dir :  template.ipynb
file/dir :  .ipynb_checkpoints
file/dir :  models
file/dir :  mnist_cams
file/dir :  mnist_pix2pix
file/dir :  tmp


### local Libraryパスを通す。

In [17]:
import sys
if os.name == 'nt':# windows
    lib_path='../colab/mylib'
else:
    print(os.getcwd())
    lib_path='/content/drive/My Drive/colab/mylib/'
sys.path.append(lib_path)

/content


### tf.data.TextLineDatasetのサンプル
https://deepage.net/tensorflow/2017/07/18/tensorflow-dataset-api.html

In [0]:
class Categories(object):
    __instance = None
    def __new__(cls, *args, **keys):
        if cls.__instance is None:
            cls.__instance = object.__new__(cls)
        return cls.__instance

    def __init__(self):
            self.items = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9' ]
            self.num   = len(self.items)

In [0]:
import datasets
import numpy as np

data = datasets.Dataset_mnist()
data.load()
input_file = data.save_for_textlinedataset(num=30)

In [0]:
def to_index(label):
    return Categories().items.index(label)

def parse_csv(line):
    [filename, category] = line.decode('utf-8').split(',')
    return filename, to_index(category)

def read_data(filename, label):
    inputs = np.load(filename.decode('utf-8')).astype(np.float32) / 255.0
    inputs = np.reshape(inputs, (inputs.shape[0], inputs.shape[1], 1))
    return inputs, label

def one_hot(data, label):
    return data, tf.one_hot( label, Categories().num  )

dataset = tf.data.TextLineDataset(input_file)
#dataset = dataset.skip(1)   # 列の読み飛ばし
if os.name == 'nt':
    dataset = dataset.map(lambda x  : tf.py_func(parse_csv, [x],    [tf.string,  tf.int32]))
    dataset = dataset.map(lambda x,y: tf.py_func(read_data, [x, y], [tf.float32, tf.int32]))
else:
    dataset = dataset.map(lambda x  : tf.py_func(parse_csv, [x],    [tf.string,  tf.int64]))
    dataset = dataset.map(lambda x,y: tf.py_func(read_data, [x, y], [tf.float32, tf.int64]))  
dataset = dataset.map(one_hot)
dataset = dataset.repeat()
dataset = dataset.shuffle(4)
dataset = dataset.batch(4)

iterator = dataset.make_one_shot_iterator()
next_elem = iterator.get_next()

In [21]:
# datasetテスト
val = sess.run(next_elem)

print(len(val))
print('\ndata ----------------------------------')
print(val[0].shape)
print(val[0].max())
print(val[0].min())
#print(val[0])

print('\nlabels ----------------------------------')
print(val[1].shape)
print(val[1])

2

data ----------------------------------
(4, 28, 28, 1)
1.0
0.0

labels ----------------------------------
(4, 10)
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]


### 訓練用モデル構築


In [22]:
import models
model = models.Model_mnist_classification()
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
imgs (InputLayer)            (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 7, 7, 128)         73856     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 7, 7, 128)         147584    
__________

### モデルの最適化

In [0]:
batch_size = 10
epochs = 10
steps_per_epoch = 3

In [24]:
history = model.fit(next_elem, epochs=epochs, steps_per_epoch=steps_per_epoch)

AttributeError: ignored

In [25]:
history = model.fit(dataset.make_one_shot_iterator(), epochs=epochs, steps_per_epoch=steps_per_epoch)

AttributeError: ignored

In [26]:
history = model.fit(dataset, epochs=epochs, steps_per_epoch=steps_per_epoch)

AttributeError: ignored

#### from_tensor_slices利用

In [27]:
x_train = data.x_train.reshape((-1,28,28,1)) / 255
y_train = keras.utils.to_categorical(data.y_train, Categories().num)
print(x_train.shape, y_train.shape)
dataset2 = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(2).repeat()

model2 = models.Model_mnist_classification_train(dataset2.make_one_shot_iterator())
model2.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
model2.fit(epochs=10, steps_per_epoch=4)

(60000, 28, 28, 1) (60000, 10)


AttributeError: ignored